In [3]:
import pandas as pd

df = pd.read_csv('./data/태양광 데이터.csv')
df

,date,0h,1h,2h,3h,4h,5h,6h,7h,8h,...,14h,15h,16h,17h,18h,19h,20h,21h,22h,23h
0,2017-10-01,0,0,0,0,0,0,0.0,0.00,0.70,...,4.1,1.70,0.90,0.7,0.5,0.10,0.0,0.0,0,0
1,2017-10-02,0,0,0,0,0,0,0.0,0.00,0.10,...,0.9,1.40,1.50,1.1,0.3,0.00,0.0,0.0,0,0
2,2017-10-03,0,0,0,0,0,0,0.0,0.00,0.10,...,3.0,1.10,1.30,0.4,0.1,0.00,0.0,0.0,0,0
3,2017-10-04,0,0,0,0,0,0,0.0,0.00,0.30,...,7.2,6.10,3.50,1.7,0.2,0.00,0.0,0.0,0,0
4,2017-10-05,0,0,0,0,0,0,0.0,0.00,2.60,...,8.3,6.20,3.30,1.5,0.4,0.00,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006,2020-07-03,0,0,0,0,0,0,0.0,0.80,2.70,...,8.3,11.41,8.59,6.9,4.9,1.50,0.5,0.0,0,0
1007,2020-07-04,0,0,0,0,0,0,0.0,0.59,2.50,...,14.4,11.70,9.60,5.9,3.7,2.30,0.8,0.0,0,0
1008,2020-07-05,0,0,0,0,0,0,0.0,0.70,2.70,...,13.3,7.00,7.20,5.6,3.5,1.79,0.5,0.0,0,0
1009,2020-07-06,0,0,0,0,0,0,0.0,0.90,2.59,...,14.1,11.70,10.10,5.6,2.8,2.40,0.5,0.0,0,0
